# FUNÇÃO USADA PARA APAGAR MARCAS NO SITE B2C

A marca só é apagada se não houver produto atrelado.

No arquivo ```.env``` editar as variáveis ```KEEP_BRAND_ID``` e ```KEEP_BRAND_NAME``` com a marca que deseja manter. Exemplo de situação:

*Suponha-se que exista marcas repetidas dentro da VTEX e desejar manter apenas 1 e apagar as outras. Primeiro, se usa a aplicação acima para trocar a marca do produto e depois executar esse programa. Nota-se que a bandMame=GIJO e brandId=2003171 é a marca que se deseja manter na VTEX e as demais apagar...*

**Foto da VTEX página de Marcas**

<img src="img/brand-list-vtex.png" alt="Marcas na VTEX B2C" >

**Portanto, preencher no arquivo ```..env``` conforme abaixo:**

```
KEEP_BRAND_ID=2003171
KEEP_BRAND_NAME=GIJO
```

**Vídeo com exemplo de extração dos dados**

<video controls>
    <source src="img/extract-vtex-brand-report.mp4" type="video/mp4">
</video>

In [ ]:
# APAGAR MARCAS

from dotenv import dotenv_values
import requests
import json
import datetime
import os
import pandas as pd

# ENV

config = dotenv_values('.env')
url_brands = config['VTEX_URL_B2C'] + '/api/catalog_system/pvt/brand/list'
url_brands_delete = config['VTEX_URL_B2C'] + '/api/catalog/pvt/brand'
headers = {
  'X-VTEX-API-AppKey': config['X_VTEX_API_APPKEY'],
  'X-VTEX-API-AppToken': config['X_VTEX_API_APPTOKEN']
}

# FUNCAO QUE CHAMA A LISTA DE MARCAS NO SITE B2C

def list_brand(url_brands, headers, data):
  try:
    response = requests.get(url_brands, headers=headers)
    response.raise_for_status()
    if(response.status_code == 200 and len(json.loads(response.text)) > 0):
      return { 'count': len(json.loads(response.text)), 'data': json.loads(response.text) }
    elif (response.status_code == 200 and len(json.loads(response.text)) == 0):
      print('Sem marcas para exibir')
    elif (response.status_code != 200):
      list_brand()
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar lista de marcas no site Kookabu\n')
    print(e.response.text)

# FUNCAO QUE DELETA A MARCA E RETORNA OK OU NOK PARA RELATORIO

def delete_brand(url_brands, headers, data):
  try:
    response = requests.delete(url_brands + '/' + str(data['id']), headers=headers)
    response.raise_for_status()
    if(response.status_code == 200):
      print('Marca {} - ID {} - APAGADA'.format(data['name'], data['id']))
      return { 'id': data['id'], 'marca': data['name'], 'status': 'OK' }
    elif (response.status_code != 200):
      delete_brand(url_brands, headers, data)
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar apagar a Marca {} - ID {} no site Kookabu\n'.format(data['name'], data['id']))
    print(e.response.text)
    return { 'id': data['id'], 'marca': data['name'], 'status': 'NOK' }

# ITERA AS MARCAS, CHAMA AS FUNCOES E AGRUPA DADOS PARA RELATORIO

brand_report = []
count = 0
content = list_brand(url_brands, headers, {})
for brand in content['data']:
  if(brand['name'] == config['KEEP_BRAND_NAME'] and str(brand['id']) != config['KEEP_BRAND_ID']):
    data = delete_brand(url_brands_delete, headers, brand)
    if(data['status'] != 'NOK'):
      brand_report.append(data)
      count += 1
    elif(data['status'] != 'OK'):
      brand_report.append(data)

# REPORT

print('CONTAGEM MARCAS APAGADAS CONFORME CRITERIO CONFIGURADO (MARCA {} - ID {})\n'.format(config['KEEP_BRAND_NAME'], config['KEEP_BRAND_ID']))
print('LIDOS -> {}\n'.format(content['count']))
print('APAGADOS -> {}\n'.format(count))
writer = pd.ExcelWriter(os.getcwd() + '/specs/report/' + datetime.datetime.now().strftime('%Y-%m-%dT%H_%M_%S-marcas-desativadas.xlsx'), engine='xlsxwriter')
pd.DataFrame(brand_report).to_excel(writer, sheet_name='brand_report')
writer.save()

# FUNÇÃO USADA PARA ATUALIZAR MARCA DO PRODUTO NO SITE B2C

Atualiza um produto com base numa lista em Excel em formato .csv ideal para trocar de forma massiva uma marca do produto.

Deve inserir o arquivo para leitura com o nome ```products.csv ``` dentro da pasta ```/specs/inputs/```. O arquivo ```products.csv ``` deve seguir o padrão:

| productId    | brandId   |
|--------------|-----------|
|15438         |2000209    |
|15437         |2000092    |

In [ ]:
# ATUALIZAR MARCA DO PRODUTO 

from dotenv import dotenv_values
import requests
import json
import datetime
import os
import pandas as pd

# ENV

def product_interface(url_products, headers, data, interface):
  if(interface == 'get'):
    try:
      response = requests.get(url_products + '/' + str(data), headers=headers)
      response.raise_for_status()
      if(response.status_code == 200):
        return json.loads(response.text)
      elif (response.status_code != 200):
        product_interface(url_products, headers, {}, 'get')
    except requests.exceptions.HTTPError as e:
      print('Erro ao chamar produto')
      print('\n{}'.format(e.response.text))
  elif(interface == 'put'):
    try:
      response = requests.put(url_products + '/' + str(data['id']), headers=headers, data=data['data'])
      response.raise_for_status()
      if(response.status_code == 200):
        return { 'productId': data['id'], 'status': 'OK', 'mensagem': 'OK' }
      elif (response.status_code != 200):
        product_interface(url_products, headers, data, 'put')
    except requests.exceptions.HTTPError as e:
      print('Erro ao atualizar o produto')
      print('\n{}'.format(e.response.text))
      return { 'productId': data['id'], 'status': 'NOK', 'mensagem': e.response.text }

config = dotenv_values('.env')
url_products = config['VTEX_URL_B2C'] + '/api/catalog/pvt/product'
headers = {
  'X-VTEX-API-AppKey': config['X_VTEX_API_APPKEY'],
  'X-VTEX-API-AppToken': config['X_VTEX_API_APPTOKEN']
}

df = pd.read_csv(os.getcwd() + '/specs/inputs/products.csv')

products_to_commit = []
count = 0
for product in df.itertuples():
  if(product['name'] == config['KEEP_BRAND_NAME'] and str(brand['id']) != config['KEEP_BRAND_ID']):
    data = product_interface(url_products, headers, product.productId, 'get')
    payload = {
    'Name': data['Name'],
    'DepartmentId': data['DepartmentId'],
    'CategoryId': data['CategoryId'],
    'BrandId': 2003171, # INSIRA O ID DA MARCA QUE DESEJA SALVAR NO PRODUTO AQUI
    'LinkId': data['LinkId'],
    'RefId': data['RefId'],
    'IsVisible': data['IsVisible'],
    'Description': data['Description'],
    'DescriptionShort': data['DescriptionShort'],
    'ReleaseDate': data['ReleaseDate'],
    'KeyWords': data['KeyWords'],
    'Title': data['Title'],
    'IsActive': data['IsActive'],
    'TaxCode': data['TaxCode'],
    'MetaTagDescription': data['MetaTagDescription'],
    'SupplierId': data['SupplierId'],
    'ShowWithoutStock': data['ShowWithoutStock'],
    'AdWordsRemarketingCode': data['AdWordsRemarketingCode'],
    'LomadeeCampaignCode': data['AdWordsRemarketingCode'],
    'Score': data['Score']
    }
    data = product_interface(url_products, headers, {'id': data['Id'], 'data': payload}, 'put')
    if(data['status'] != 'NOK'):
      products_to_commit.append(data)
      count += 1
    elif(data['status'] != 'OK'):
      products_to_commit.append(data)

print('CONTAGEM DE PRODUTOS PARA ATUALIZAR -> {}\n'.format(len(df)))
print('PROCESSADOS -> {}\n'.format(count))
writer = pd.ExcelWriter(os.getcwd() + '/specs/report/' + datetime.datetime.now().strftime('%Y-%m-%dT%H_%M_%S-produtos-atualizados.xlsx'), engine='xlsxwriter')
pd.DataFrame(products_to_commit).to_excel(writer, sheet_name='product_report')
writer.save()

# FUNÇÃO USADA PARA INATIVAR MARCA NÃO ELEGÍVEL PARA O SITE B2C 

Esse programa busca todas as marcas na VTEX B2C e inativa a marca caso ela não seja uma das elegíveis abaixo:

|Marca         |brandId    |
|--------------|-----------|
|Alakazoo	       |2001063|
|Amora	           |2000214|
|Baby best	       |2001032|
|Baby Gijo	       |2000115|
|Brandili 	       |2000137|
|Color Girl	       |2000092|
|Color Mini 	   |2000197|
|Din Don  	       |2000083|
|Galzerano	       |2001196|
|Gijo	           |2003171|
|Gijo Kids 	       |2000187|
|Kiki Xodo	       |2000271|
|Kiko Baby	       |2000246|
|Kookabu	       |2000129|
|Kyly	           |2000305|
|Lancher	       |2000370|
|Linha Praia Everly|2000078|
|Menina Anjo       |2000434|
|Milon 	           |2000208|
|Nanai 	           |2000382|
|Okyside           |2000123|
|Pami 	           |2000194|
|Smoby Baby        |2000041|

In [ ]:
# INATIVAR MARCAS NÃO ELEGÍVEIS PARA O SITE B2C

from dotenv import dotenv_values
import requests
import json
import datetime
import os
import pandas as pd

# FUNCAO QUE CHAMA A LISTA DE MARCAS NO SITE B2C

def list_brand(url_brands, headers, data):
  try:
    response = requests.get(url_brands, headers=headers)
    response.raise_for_status()
    if(response.status_code == 200 and len(json.loads(response.text)) > 0):
      return { 'count': len(json.loads(response.text)), 'data': json.loads(response.text) }
    elif (response.status_code == 200 and len(json.loads(response.text)) == 0):
      print('Sem marcas para exibir')
    elif (response.status_code != 200):
      list_brand(url_brands, headers, {})
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar lista de marcas no site Kookabu\n')
    print(e.response.text)

# FUNCAO QUE DELETA A MARCA E RETORNA OK OU NOK PARA RELATORIO

def update_brand(url_brands, headers, data):
  try:
    response = requests.put(url_brands + '/' + str(data['Id']), headers=headers, data=data)
    response.raise_for_status()
    if(response.status_code == 200):
      return { 'brandId': data['Id'], 'marca': data['Name'], 'status': 'OK' }
    elif (response.status_code != 200):
      update_brand(url_brands, headers, data)
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar atualizar a Marca {} - ID {} no site Kookabu\n'.format(data['Name'], data['Id']))
    print(e.response.text)
    return { 'brandId': data['Id'], 'marca': data['Name'], 'status': e.response.text }

# ITERA AS MARCAS, CHAMA AS FUNCOES E AGRUPA DADOS PARA RELATORIO


config = dotenv_values('.env')
url_brands = config['VTEX_URL_B2C'] + '/api/catalog_system/pvt/brand/list'
url_brands_update = config['VTEX_URL_B2C'] + '/api/catalog/pvt/brand'
headers = {
  'X-VTEX-API-AppKey': config['X_VTEX_API_APPKEY'],
  'X-VTEX-API-AppToken': config['X_VTEX_API_APPTOKEN']
}

brand_report = []
count = 0
content = list_brand(url_brands, headers, {})
for brand in content['data']:
  if(int(brand['id']) != 2001063 or int(brand['id']) != 2000305 or int(brand['id']) != 2000208 or int(brand['id']) != 2000382 or int(brand['id']) != 2000214 or int(brand['id']) != 2000271 or int(brand['id']) != 2000434 or int(brand['id']) != 2000078 or int(brand['id']) != 2000246 or int(brand['id']) != 2000137 or int(brand['id']) != 2000370 or int(brand['id']) != 2000129 or int(brand['id']) != 2000083 or int(brand['id']) != 2000115 or int(brand['id']) != 2000187 or int(brand['id']) != 2000197 or int(brand['id']) != 2000092 or int(brand['id']) != 2000194 or int(brand['id']) != 2000123 or int(brand['id']) != 2003171 or int(brand['id']) != 2000041 or int(brand['id']) != 2001196 or int(brand['id']) != 2001032):
    payload = {
      'Id': brand['id'],
      'Name': brand['name'],
      'Text': brand['name'],
      'Keywords': None,
      'SiteTitle': None,
      'Active': False,
      'MenuHome': True,
      'AdWordsRemarketingCode': None,
      'LomadeeCampaignCode': None,
      'Score': None,
      'LinkId': brand['name']
    }
    data = update_brand(url_brands_update, headers, payload)
    if(data['status'] != 'NOK'):
      brand_report.append(data)
      count += 1
    elif(data['status'] != 'OK'):
      brand_report.append(data)
  if(count % 100 == 0):
    print('PROGRESSO {}/{}\n'.format(count, content['count']))

# REPORT

print('LIDOS -> {}\n'.format(content['count']))
print('ATUALIZADOS -> {}\n'.format(count))
writer = pd.ExcelWriter(os.getcwd() + '/specs/report/' + datetime.datetime.now().strftime('%Y-%m-%dT%H_%M_%S-marcas-atualizadas-inativas.xlsx'), engine='xlsxwriter')
pd.DataFrame(brand_report).to_excel(writer, sheet_name='brand_report_update')
writer.save()

# FUNÇÃO USADA PARA ATIVAR MARCA ELEGÍVEL PARA O SITE B2C 

Esse programa busca todas as marcas na VTEX B2C e ativa a marca elegível:

|Marca             |brandId    |
|------------------|-----------|
|Alakazoo	       |2001063    |
|Amora	           |2000214    |
|Baby best	       |2001032    |
|Baby Gijo	       |2000115    |
|Brandili 	       |2000137    |
|Color Girl	       |2000092    |
|Color Mini 	   |2000197    |
|Din Don  	       |2000083    |
|Galzerano	       |2001196    |
|Gijo	           |2003171    |
|Gijo Kids 	       |2000187    |
|Kiki Xodo	       |2000271    |
|Kiko Baby	       |2000246    |
|Kookabu	       |2000129    |
|Kyly	           |2000305    |
|Lancher	       |2000370    |
|Linha Praia Everly|2000078    |
|Menina Anjo       |2000434    |
|Milon 	           |2000208    |
|Nanai 	           |2000382    |
|Okyside           |2000123    |
|Pami 	           |2000194    |
|Smoby Baby        |2000041    |

In [ ]:
# ATIVAR MARCAS ELEGÍVEIS PARA O SITE B2C

from dotenv import dotenv_values
import requests
import json
import datetime
import os
import pandas as pd

# FUNCAO QUE CHAMA A LISTA DE MARCAS NO SITE B2C

def list_brand(url_brands, headers, data):
  try:
    response = requests.get(url_brands, headers=headers)
    response.raise_for_status()
    if(response.status_code == 200 and len(json.loads(response.text)) > 0):
      return { 'count': len(json.loads(response.text)), 'data': json.loads(response.text) }
    elif (response.status_code == 200 and len(json.loads(response.text)) == 0):
      print('Sem marcas para exibir')
    elif (response.status_code != 200):
      list_brand(url_brands, headers, {})
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar lista de marcas no site Kookabu\n')
    print(e.response.text)

# FUNCAO QUE ATUALIZA A MARCA E RETORNA OK OU NOK PARA RELATORIO

def update_brand(url_brands, headers, data):
  try:
    response = requests.put(url_brands + '/' + str(data['Id']), headers=headers, data=data)
    response.raise_for_status()
    if(response.status_code == 200):
      return { 'brandId': data['Id'], 'marca': data['Name'], 'status': 'OK' }
    elif (response.status_code != 200):
      update_brand(url_brands, headers, data)
  except requests.exceptions.HTTPError as e:
    print('Erro ao chamar atualizar a Marca {} - ID {} no site Kookabu\n'.format(data['Name'], data['Id']))
    print(e.response.text)
    return { 'brandId': data['Id'], 'marca': data['Name'], 'status': e.response.text }

# ITERA AS MARCAS, CHAMA AS FUNCOES E AGRUPA DADOS PARA RELATORIO

config = dotenv_values('.env')
url_brands = config['VTEX_URL_B2C'] + '/api/catalog_system/pvt/brand/list'
url_brands_update = config['VTEX_URL_B2C'] + '/api/catalog/pvt/brand'
headers = {
  'X-VTEX-API-AppKey': config['X_VTEX_API_APPKEY'],
  'X-VTEX-API-AppToken': config['X_VTEX_API_APPTOKEN']
}

brand_report = []
count = 0
content = list_brand(url_brands, headers, {})
for brand in content['data']:
  if(int(brand['id']) == 2001063 or int(brand['id']) == 2000305 or int(brand['id']) == 2000208 or int(brand['id']) == 2000382 or int(brand['id']) == 2000214 or int(brand['id']) == 2000271 or int(brand['id']) == 2000434 or int(brand['id']) == 2000078 or int(brand['id']) == 2000246 or int(brand['id']) == 2000137 or int(brand['id']) == 2000370 or int(brand['id']) == 2000129 or int(brand['id']) == 2000083 or int(brand['id']) == 2000115 or int(brand['id']) == 2000187 or int(brand['id']) == 2000197 or int(brand['id']) == 2000092 or int(brand['id']) == 2000194 or int(brand['id']) == 2000123 or int(brand['id']) == 2003171 or int(brand['id']) == 2000041 or int(brand['id']) == 2001196 or int(brand['id']) == 2001032):
    payload = {
      'Id': brand['id'],
      'Name': brand['name'],
      'Text': brand['name'],
      'Keywords': None,
      'SiteTitle': None,
      'Active': True,
      'MenuHome': True,
      'AdWordsRemarketingCode': None,
      'LomadeeCampaignCode': None,
      'Score': None,
      'LinkId': brand['name']
    }
    data = update_brand(url_brands_update, headers, payload)
    if(data['status'] != 'NOK'):
      brand_report.append(data)
      count += 1
    elif(data['status'] != 'OK'):
      brand_report.append(data)
      count += 1
  if(count % 200 == 0):
    print('{} - PROGRESSO {}/{}\n'.format(datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S'), count, content['count']))

# REPORT

print('LIDOS -> {}\n'.format(content['count']))
print('ATUALIZADOS -> {}\n'.format(count))
writer = pd.ExcelWriter(os.getcwd() + '/specs/report/' + datetime.datetime.now().strftime('%Y-%m-%dT%H_%M_%S-marcas-atualizadas-ativas.xlsx'), engine='xlsxwriter')
pd.DataFrame(brand_report).to_excel(writer, sheet_name='brand_report_update')
writer.save()

# FUNÇÃO USADA PARA MANTER APENAS OS PRODUTOS DAS MARCAS ELEGÍVEIS ATIVOS NO SITE

Esse programa busca na planilha ```productIds.csv``` todos produtos exportados da VTEX e faz a atualização do mesmo para manter apenas os elegíveis no site B2C. Apenas os produtos das marcas a seguir são os elegíveis para se manterem ativos no site B2C:

|Marca             |brandId    |
|------------------|-----------|
|Alakazoo	       |2001063    |
|Amora	           |2000214    |
|Baby best	       |2001032    |
|Baby Gijo	       |2000115    |
|Brandili 	       |2000137    |
|Color Girl	       |2000092    |
|Color Mini 	   |2000197    |
|Din Don  	       |2000083    |
|Galzerano	       |2001196    |
|Gijo	           |2003171    |
|Gijo Kids 	       |2000187    |
|Kiki Xodo	       |2000271    |
|Kiko Baby	       |2000246    |
|Kookabu	       |2000129    |
|Kyly	           |2000305    |
|Lancher	       |2000370    |
|Linha Praia Everly|2000078    |
|Menina Anjo       |2000434    |
|Milon 	           |2000208    |
|Nanai 	           |2000382    |
|Okyside           |2000123    |
|Pami 	           |2000194    |
|Smoby Baby        |2000041    |

Salvar na pasta ```/specs/inputs/productIds.csv``` conforme exemplo do vídeo a seguir:

<video controls>
    <source src="img/extract-vtex-product-report.mp4" type="video/mp4">
</video>

In [ ]:
# ATUALIZAR PRODUTO

from dotenv import dotenv_values
import requests
import json
import datetime
import os
import pandas as pd
import time

# ENV

def product_interface(url_products, headers, data, interface):
  if(interface == 'get'):
    try:
      response = requests.get(url_products + '/' + str(data), headers=headers)
      response.raise_for_status()
      if(response.status_code == 200):
        return json.loads(response.text)
      elif (response.status_code != 200):
        product_interface(url_products, headers, {}, 'get')
    except requests.exceptions.HTTPError as e:
      print('Erro ao chamar produto')
      print('\n{}'.format(e.response.text))
  elif(interface == 'put'):
    try:
      response = requests.put(url_products + '/' + str(data['id']), headers=headers, data=data['data'])
      response.raise_for_status()
      if(response.status_code == 200):
        return { 'productId': data['id'], 'status': 'OK', 'mensagem': 'OK' }
      elif (response.status_code != 200):
        product_interface(url_products, headers, data, 'put')
    except requests.exceptions.HTTPError as e:
      print('Erro ao atualizar o produto')
      print('\n{}'.format(e.response.text))
      return { 'productId': data['id'], 'status': 'NOK', 'mensagem': e.response.text }

config = dotenv_values('.env')
url_products = config['VTEX_URL_B2C'] + '/api/catalog/pvt/product'
headers = {
  'X-VTEX-API-AppKey': config['X_VTEX_API_APPKEY'],
  'X-VTEX-API-AppToken': config['X_VTEX_API_APPTOKEN']
}

df = pd.read_csv(os.getcwd() + '/specs/inputs/productIds.csv')
df = df.drop_duplicates()

# ATUALIZACAO PRODUTO

products_to_commit = []
count = 0
for product in df.itertuples():
  data = product_interface(url_products, headers, product.productId, 'get')
  if(data['BrandId'] != 2001063 or data['BrandId'] != 2000305 or data['BrandId'] != 2000208 or data['BrandId'] != 2000382 or data['BrandId'] != 2000214 or data['BrandId'] != 2000271 or data['BrandId'] != 2000434 or data['BrandId'] != 2000078 or data['BrandId'] != 2000246 or data['BrandId'] != 2000137 or data['BrandId'] != 2000370 or data['BrandId'] != 2000129 or data['BrandId'] != 2000083 or data['BrandId'] != 2000115 or data['BrandId'] != 2000187 or data['BrandId'] != 2000197 or data['BrandId'] != 2000092 or data['BrandId'] != 2000194 or data['BrandId'] != 2000123 or data['BrandId'] != 2003171 or data['BrandId'] != 2000041 or data['BrandId'] != 2001196 or data['BrandId'] != 2001032):
    payload = {
    'Name': data['Name'],
    'DepartmentId': data['DepartmentId'],
    'CategoryId': data['CategoryId'],
    'BrandId': data['BrandId'],
    'LinkId': data['LinkId'],
    'RefId': data['RefId'],
    'IsVisible': data['IsVisible'],
    'Description': data['Description'],
    'DescriptionShort': data['DescriptionShort'],
    'ReleaseDate': data['ReleaseDate'],
    'KeyWords': data['KeyWords'],
    'Title': data['Title'],
    'IsActive': False,
    'TaxCode': data['TaxCode'],
    'MetaTagDescription': data['MetaTagDescription'],
    'SupplierId': data['SupplierId'],
    'ShowWithoutStock': data['ShowWithoutStock'],
    'AdWordsRemarketingCode': data['AdWordsRemarketingCode'],
    'LomadeeCampaignCode': data['AdWordsRemarketingCode'],
    'Score': data['Score']
    }
  elif(data['BrandId'] == 2001063 or data['BrandId'] == 2000305 or data['BrandId'] == 2000208 or data['BrandId'] == 2000382 or data['BrandId'] == 2000214 or data['BrandId'] == 2000271 or data['BrandId'] == 2000434 or data['BrandId'] == 2000078 or data['BrandId'] == 2000246 or data['BrandId'] == 2000137 or data['BrandId'] == 2000370 or data['BrandId'] == 2000129 or data['BrandId'] == 2000083 or data['BrandId'] == 2000115 or data['BrandId'] == 2000187 or data['BrandId'] == 2000197 or data['BrandId'] == 2000092 or data['BrandId'] == 2000194 or data['BrandId'] == 2000123 or data['BrandId'] == 2003171 or data['BrandId'] == 2000041 or data['BrandId'] == 2001196 or data['BrandId'] == 2001032):
    payload = {
    'Name': data['Name'],
    'DepartmentId': data['DepartmentId'],
    'CategoryId': data['CategoryId'],
    'BrandId': data['BrandId'],
    'LinkId': data['LinkId'],
    'RefId': data['RefId'],
    'IsVisible': data['IsVisible'],
    'Description': data['Description'],
    'DescriptionShort': data['DescriptionShort'],
    'ReleaseDate': data['ReleaseDate'],
    'KeyWords': data['KeyWords'],
    'Title': data['Title'],
    'IsActive': True,
    'TaxCode': data['TaxCode'],
    'MetaTagDescription': data['MetaTagDescription'],
    'SupplierId': data['SupplierId'],
    'ShowWithoutStock': data['ShowWithoutStock'],
    'AdWordsRemarketingCode': data['AdWordsRemarketingCode'],
    'LomadeeCampaignCode': data['AdWordsRemarketingCode'],
    'Score': data['Score']
    }
  data = product_interface(url_products, headers, {'id': data['Id'], 'data': payload}, 'put')
  if(data['status'] != 'NOK'):
    products_to_commit.append(data)
    count += 1
  elif(data['status'] != 'OK'):
    products_to_commit.append(data)
  if(count % 1000 == 0):
    print('PROGRESSO {}/{} - {}\n'.format(count, len(df), datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')))
    time.sleep(60)

print('CONTAGEM DE PRODUTOS PARA ATUALIZAR -> {}\n'.format(len(df)))
print('PROCESSADOS -> {}\n'.format(count))
writer = pd.ExcelWriter(os.getcwd() + '/specs/report/' + datetime.datetime.now().strftime('%Y-%m-%dT%H_%M_%S-produtos-atualizados.xlsx'), engine='xlsxwriter')
pd.DataFrame(products_to_commit).to_excel(writer, sheet_name='product_report')
writer.save()